<a href="https://colab.research.google.com/github/sudhir2016/Google-Colab-12/blob/master/Fruit_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Python, TensorFlow and Keras libraries.

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.image as im
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
import cv2
import glob
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import load_model

Load Fruit Images dataset.

In [0]:
!git clone https://github.com/Horea94/Fruit-Images-Dataset

Create empty lists for fruit images (X) and labels(Y) for testing.

In [0]:
X=[]
Y=[]

Create and sort list of label names.

In [0]:
label=[]

In [0]:
for name in glob.glob('/content/Fruit-Images-Dataset/Training/*'):
  name1=name[39:]
  label.append(name1)

In [0]:
print(label)

In [0]:
len(label)

In [0]:
label.sort()

In [0]:
print(label)

Path for fruit dataset.

In [0]:
a='/content/Fruit-Images-Dataset/Training'
c='*.*'

Load and preprocess small set of 50 images, 5 of each category for test dataset.

In [0]:
for i in range (131):
  b=label[i]
  path=os.path.join(a,b,c)
  image1= [im.imread(file) for file in glob.glob(path)] 
  for j in range (5):
    image2 = cv2.resize(image1[j], dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    X.append(image2)
    out=i
    Y.append(out)

In [0]:
len(X)

In [0]:
X=np.array(X)/255.0

In [0]:
Y=np.array(Y)

Set up train and validation dataset generators as flow from directory.

In [0]:
train_datagen=ImageDataGenerator(rescale=1./255)

In [21]:
train_generator=train_datagen.flow_from_directory('/content/Fruit-Images-Dataset/Training',target_size=(100,100),
        batch_size=32,
        class_mode='sparse')

Found 67692 images belonging to 131 classes.


In [0]:
validation_datagen=ImageDataGenerator(rescale=1./255)

In [0]:
validation_generator=validation_datagen.flow_from_directory('/content/Fruit-Images-Dataset/Test',target_size=(100, 100),
        batch_size=32,
        class_mode='sparse')

In [0]:
print(train_generator.class_indices)

Build the model

In [0]:
baseModel = VGG16(weights="imagenet", include_top=False,input_shape=(100,100,3))
	

In [0]:
for layer in baseModel.layers:
	layer.trainable = False

In [0]:
headModel = baseModel.output
headModel = GlobalAveragePooling2D() (headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dense(131, activation="softmax")(headModel)

In [0]:
model = Model(inputs=baseModel.input, outputs=headModel)

Compile the model


In [0]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.summary()

Train the model

In [0]:
model.fit(train_generator,
        steps_per_epoch=2000,
        epochs=2,validation_data=validation_generator,validation_steps=600)

Evaluate the model

In [0]:
model.evaluate(X,Y)

Make predictions

In [0]:
p=model.predict(X)

Verify prediction

Make a selection

In [0]:
s=200
pred=p[s]

In [0]:
m1=np.argmax(pred)
l1=label[m1]
print('prediction :',l1)

In [0]:
m2=Y[s]
l2=label[m2]
print('actual :',l2)

In [0]:
plt.imshow(X[s])